In [3]:
#Tugas KNN dengan public data
#Cars Data has Information about 3 brands/make of cars. Namely US, Japan, Europe. 
#Target of the data set to find the brand of a car using the parameters such as horsepower, Cubic inches, Make year, etc.
#create a predictive data model to predict the car brand.
#sumber data : https://www.kaggle.com/abineshkumark/carsdata

import pandas as pd
import numpy as np

#input data
data_source=pd.read_csv("data\cars.csv")
#copy data
data = data_source.copy(deep = True)

data.head()

,mpg,cylinders,cubicinches,hp,weightlbs,time-to-60,year,brand
0,14.0,8,350.0,165,4209.0,12,1972,US.
1,31.9,4,89.0,71,1925.0,14,1980,Europe.
2,17.0,8,302.0,140,3449.0,11,1971,US.
3,15.0,8,400.0,150,3761.0,10,1971,US.
4,30.5,4,98.0,63,2051.0,17,1978,US.


In [4]:
#konversi class ke tipe diskrit
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data.brand=le.fit_transform(data.brand) #Europe. :0, Japan.:1, US.:2
print(data)

#data memiliki 261 baris data dan 8 kolom, dengan varibel terget brand dengan 3 class yaiut Europe. :0, Japan.:1, US.:2
data.info()

#deskirpsi data statistik, dan tidak terdapat data noisy
data.describe().T

      mpg  cylinders  cubicinches   hp  weightlbs  time-to-60  year  brand
0    14.0          8        350.0  165     4209.0          12  1972      2
1    31.9          4         89.0   71     1925.0          14  1980      0
2    17.0          8        302.0  140     3449.0          11  1971      2
3    15.0          8        400.0  150     3761.0          10  1971      2
4    30.5          4         98.0   63     2051.0          17  1978      2
..    ...        ...          ...  ...        ...         ...   ...    ...
256  17.0          8        305.0  130     3840.0          15  1980      2
257  36.1          4         91.0   60     1800.0          16  1979      1
258  22.0          6        232.0  112     2835.0          15  1983      2
259  18.0          6        232.0  100     3288.0          16  1972      2
260  22.0          6        250.0  105     3353.0          15  1977      2

[261 rows x 8 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data

,count,mean,std,min,25%,50%,75%,max
mpg,261.0,23.144828,7.823570,10.0,16.9,22.0,28.8,46.6
cylinders,261.0,5.590038,1.733310,3.0,4.0,6.0,8.0,8.0
cubicinches,259.0,200.918919,109.258305,68.0,99.5,156.0,303.0,455.0
hp,261.0,106.360153,40.499959,46.0,75.0,95.0,138.0,230.0
weightlbs,258.0,3009.833333,854.039019,1613.0,2248.0,2867.5,3670.0,4997.0
time-to-60,261.0,15.547893,2.910625,8.0,14.0,16.0,17.0,25.0
year,261.0,1976.819923,3.637696,1971.0,1974.0,1977.0,1980.0,1983.0
brand,261.0,1.436782,0.784972,0.0,1.0,2.0,2.0,2.0


In [5]:
#melihat jumlah data null pada datasets
data.isnull().sum()

mpg            0
cylinders      0
cubicinches    2
hp             0
weightlbs      3
time-to-60     0
year           0
brand          0
dtype: int64

In [6]:
#terdapat beberapa data yang null('cubicinches','weightlbs'), kemudian update data null dengan nilai median
data['cubicinches'].fillna(data['cubicinches'].median(), inplace = True)
data['weightlbs'].fillna(data['weightlbs'].median(), inplace = True)
data.isnull().sum()

mpg            0
cylinders      0
cubicinches    0
hp             0
weightlbs      0
time-to-60     0
year           0
brand          0
dtype: int64

In [7]:
#Scaling, standarisasi data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
y = data.brand
x = pd.DataFrame(sc.fit_transform(data.drop(["brand"], axis = 1),),columns=['mpg','cylinders','cubicinches','hp','weightlbs','time-to-60','year'])
x.head()

,mpg,cylinders,cubicinches,hp,weightlbs,time-to-60,year
0,-1.171127,1.393053,1.374668,1.450681,1.416700,-1.221287,-1.327539
1,1.121226,-0.919105,-1.026454,-0.874768,-1.277950,-0.532829,0.875880
2,-0.786934,1.393053,0.933082,0.832210,0.520057,-1.565516,-1.602966
3,-1.043063,1.393053,1.834653,1.079598,0.888153,-1.909745,-1.602966
4,0.941936,-0.919105,-0.943657,-1.072679,-1.129296,0.499857,0.325025


In [8]:
#print jumlah class pada datasets
print()
print("Jumlah class pada datasets")
print("--------------------------------")
print(y.value_counts())

#Resampling data, untuk keseimbangan data class
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=19)

x = pd.concat([x_train, y_train], axis=1)
europe = x[x.brand==0]
japan = x[x.brand==1]
us = x[x.brand==2]
#menaikan class minoritas
from sklearn.utils import resample
japan_upsampled=resample(japan, replace=True, n_samples=len(us), random_state=19)
europe_upsampled=resample(europe, replace=True, n_samples=len(us), random_state=19)

#gabungkan
upsampled = pd.concat([us, japan_upsampled, europe_upsampled])

#print jumlah class pada datatraining setelah resampling
print()
print("Jumlah class pada datatraining setelah resampling")
print("--------------------------------")
print(upsampled.brand.value_counts())


Jumlah class pada datasets
--------------------------------
2    162
1     51
0     48
Name: brand, dtype: int64

Jumlah class pada datatraining setelah resampling
--------------------------------
2    130
1    130
0    130
Name: brand, dtype: int64


In [9]:
#membagi data training dan data testing yang telah resampling
dt_train = upsampled
dt_test = pd.concat([x_test, y_test], axis=1)
dt_all = dt_train.append(dt_test)

x = dt_all[['mpg','cylinders','cubicinches','hp','weightlbs','time-to-60','year']]
y = dt_all.brand

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=30, stratify=y)
print()
print("Jumlah dataset setelah di-split :")
print("--------------------------")
print("data training : ",y_train.count())
print("data test : ",y_test.count())
print()


Jumlah dataset setelah di-split :
--------------------------
data training :  354
data test :  89



In [10]:
#menggunkan KKN
from sklearn.neighbors import KNeighborsClassifier
test_scores = []
train_scores = []

for i in range(1,18):
    knn = KNeighborsClassifier(i)
    knn.fit(x_train, y_train)
    
    train_scores.append(knn.score(x_train, y_train))
    test_scores.append(knn.score(x_test, y_test))

print("Train Scores :", end="")
print(train_scores)
print()
print("Test Scores :", end="")
print(test_scores)

Train Scores :[1.0, 0.9350282485875706, 0.9293785310734464, 0.8926553672316384, 0.8757062146892656, 0.8785310734463276, 0.867231638418079, 0.8587570621468926, 0.8587570621468926, 0.867231638418079, 0.844632768361582, 0.8305084745762712, 0.8389830508474576, 0.8192090395480226, 0.8333333333333334, 0.8107344632768362, 0.8248587570621468]

Test Scores :[0.9213483146067416, 0.8876404494382022, 0.898876404494382, 0.8314606741573034, 0.8089887640449438, 0.8089887640449438, 0.8089887640449438, 0.8202247191011236, 0.8089887640449438, 0.8314606741573034, 0.7865168539325843, 0.7865168539325843, 0.8089887640449438, 0.7752808988764045, 0.7752808988764045, 0.7752808988764045, 0.7865168539325843]


In [11]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
#lakukan prediksi
y_pred = knn.predict(x_test)
confusion_matrix(y_test, y_pred)
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,2,All
True,,,,
0,25,3,0,28
1,8,19,1,28
2,2,5,26,33
All,35,27,27,89


In [12]:
#Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.89      0.79        28
           1       0.70      0.68      0.69        28
           2       0.96      0.79      0.87        33

    accuracy                           0.79        89
   macro avg       0.79      0.79      0.78        89
weighted avg       0.80      0.79      0.79        89

